In [1]:
!pip install earthaccess python-dateutil

In [12]:
# ecostress_lst_earthaccess.py
import os
import math
from urllib.parse import urlparse
from dateutil.parser import isoparse
import earthaccess as ea

# ==========================
# User inputs (edit these)
# ==========================
LAT = 38.92744661
LON = -106.95647367
SEARCH_RADIUS_KM = 1

START_DATE = "2017-08-16"
END_DATE   = "2017-08-18"

# Products:
#   "ECO_L2G_LSTE" (gridded LST&E, 70 m)  or  "ECO_L2T_LSTE" (tiled)
PRODUCT_SHORT_NAME = "ECO_L2G_LSTE"
PRODUCT_VERSION = "002"

OUT_DIR = "./ecostress_downloads"

# ==========================
# Helpers
# ==========================
def km_to_deg_lat(km: float) -> float:
    return km / 110.574

def km_to_deg_lon(km: float, lat_deg: float) -> float:
    return km / (111.320 * math.cos(math.radians(lat_deg)))

def point_radius_to_bbox(lon: float, lat: float, radius_km: float):
    dlat = km_to_deg_lat(radius_km)
    dlon = km_to_deg_lon(radius_km, lat)
    min_lon = max(-180.0, lon - dlon)
    max_lon = min( 180.0, lon + dlon)
    min_lat = max( -90.0, lat - dlat)
    max_lat = min(  90.0, lat + dlat)
    return (min_lon, min_lat, max_lon, max_lat)

def normalize_temporal(start_date: str, end_date: str):
    def to_iso(s, is_start):
        if "T" in s:
            return isoparse(s).strftime("%Y-%m-%dT%H:%M:%SZ")
        return f"{s}T00:00:00Z" if is_start else f"{s}T23:59:59Z"
    return (to_iso(start_date, True), to_iso(end_date, False))

def first_direct_link(g):
    """Return the first HTTPS direct data link (not OPeNDAP) if available."""
    try:
        # earthaccess Granule has .data_links() method
        links = g.data_links(access="direct")
    except Exception:
        # some versions expose a list property
        links = getattr(g, "data_links", []) or []
    for href in links:
        h = str(href)
        if h.startswith("https://") and "opendap" not in h.lower():
            return h
    return None

def filename_from_link(href: str) -> str:
    if not href:
        return "UNKNOWN_FILENAME"
    return os.path.basename(urlparse(href).path)

def safe_umm(g):
    # Works whether g.umm exists or g is a dict-like
    if hasattr(g, "umm"):
        return getattr(g, "umm") or {}
    try:
        return g.get("umm", {})  # type: ignore[attr-defined]
    except Exception:
        return {}

def safe_temporal(umm: dict):
    try:
        r = umm.get("TemporalExtent", {}).get("RangeDateTime", {})
        return r.get("BeginningDateTime", "N/A"), r.get("EndingDateTime", "N/A")
    except Exception:
        return "N/A", "N/A"

def safe_size_mb(umm: dict):
    try:
        adi = umm.get("DataGranule", {}).get("ArchiveAndDistributionInformation", [])
        if adi and "SizeMBDataGranule" in adi[0]:
            return float(adi[0]["SizeMBDataGranule"])
    except Exception:
        pass
    return None

def print_granule_list(results):
    print("\nFound granules:")
    for i, g in enumerate(results, start=1):
        href = first_direct_link(g)
        fname = filename_from_link(href)
        umm = safe_umm(g)
        t0, t1 = safe_temporal(umm)
        size = safe_size_mb(umm)
        size_str = f"{size:.1f} MB" if size is not None else "N/A"
        print(f"[{i:02d}] {fname} | {t0} → {t1} | Size: {size_str}")

# ==========================
# Main
# ==========================
def main():
    print("Logging in to Earthdata (interactive)…")
    ea.login(strategy="interactive")

    bbox = point_radius_to_bbox(LON, LAT, SEARCH_RADIUS_KM)
    t0, t1 = normalize_temporal(START_DATE, END_DATE)

    print(f"\nSearching {PRODUCT_SHORT_NAME} v{PRODUCT_VERSION}")
    print(f"Spatial bbox: {bbox}")
    print(f"Temporal: {t0} to {t1}")

    results = ea.search_data(
        short_name=PRODUCT_SHORT_NAME,
        version=PRODUCT_VERSION,
        temporal=(t0, t1),
        bounding_box=bbox,
        provider="LPCLOUD",
        sort_key="-start_date",
    )

    if not results:
        print("No matching granules found.")
        return

    print_granule_list(results)

    sel = input("\nEnter the index of the granule to download: ").strip()
    try:
        idx = int(sel) - 1
        assert 0 <= idx < len(results)
    except Exception:
        print("Invalid selection.")
        return

    chosen = [results[idx]]
    os.makedirs(OUT_DIR, exist_ok=True)

    print("\nDownloading…")
    paths = ea.download(chosen, OUT_DIR)

    if paths:
        print("Download complete:")
        for p in paths:
            print("  -", p)
    else:
        print("Download returned no files. If you’re on Windows, ensure your browser login succeeded and try again.")

if __name__ == "__main__":
    main()


Logging in to Earthdata (interactive)…

Searching ECO_L2G_LSTE v002
Spatial bbox: (-106.96802094336361, 38.91840289267043, -106.94492639663638, 38.936490327329565)
Temporal: 2017-08-16T00:00:00Z to 2017-08-18T23:59:59Z
No matching granules found.


In [9]:
# ecostress_h5_to_geotiff_gdal_only.py
import numpy as np
from osgeo import gdal

H5_PATH = r"ecostress_downloads\ECOv002_L2G_LSTE_39343_012_20250614T104308_0713_01.h5"
OUT_TIF_C = r"ecostress_downloads\lst_celsius.tif"

# 1) Find the LST subdataset
g = gdal.Open(H5_PATH, gdal.GA_ReadOnly)
if g is None:
    raise RuntimeError("GDAL could not open the HDF5 file.")
subs = g.GetSubDatasets()

lst_sds = None
for name, desc in subs:
    low = (name + " " + desc).lower()
    if "/lst" in low and "qc" not in low:  # pick LST, not QC
        lst_sds = name
        break
if not lst_sds:
    raise RuntimeError("LST subdataset not found. Run gdalinfo on the .h5 and share the output.")

# 2) Open LST subdataset and read data (Kelvin)
src = gdal.Open(lst_sds, gdal.GA_ReadOnly)
arr_k = src.ReadAsArray().astype("float32")

# 3) Mask fill values (common ECOSTRESS fill)
for fv in (-9999.0, -9998.0):
    arr_k[arr_k == fv] = np.nan

# 4) Convert to Celsius
arr_c = arr_k - 273.15

# 5) Write GeoTIFF with same georeferencing
driver = gdal.GetDriverByName("GTiff")
dst = driver.Create(
    OUT_TIF_C,
    src.RasterXSize,
    src.RasterYSize,
    1,
    gdal.GDT_Float32,
    options=["TILED=YES", "COMPRESS=DEFLATE"]
)
dst.SetGeoTransform(src.GetGeoTransform())
dst.SetProjection(src.GetProjection())
band = dst.GetRasterBand(1)
band.WriteArray(arr_c)
band.SetNoDataValue(np.nan)
band.FlushCache()
dst = None
src = None
print("Wrote:", OUT_TIF_C)


Wrote: ecostress_downloads\lst_celsius.tif


In [3]:
import os
from osgeo import gdal

gdal.UseExceptions()

H5_PATH = r"ecostress_downloads\ECOv002_L2G_LSTE_39606_011_20250701T045755_0713_01.h5"  # adjust if needed

print("Resolved path:", os.path.abspath(H5_PATH))
print("Exists?      :", os.path.exists(H5_PATH))
print("Size (MB)    :", round(os.path.getsize(H5_PATH)/1e6, 3) if os.path.exists(H5_PATH) else "N/A")

# 1) Is the HDF5 driver available?
drv = gdal.GetDriverByName("HDF5")
print("HDF5 driver present?", bool(drv))

# 2) Try opening the container with explicit drivers
try:
    ds = gdal.OpenEx(H5_PATH, gdal.OF_READONLY | gdal.OF_RASTER, allowed_drivers=["HDF5","HDF5Image"])
    print("OpenEx('HDF5','HDF5Image') ds is None?", ds is None)
    if ds:
        print("Subdatasets:", ds.GetSubDatasets()[:3], "… total:", len(ds.GetSubDatasets()))
except RuntimeError as e:
    print("OpenEx error:", e)

# 3) Try opening the LST subdataset path directly (common for ECOSTRESS L2G v002)
try:
    sds_path = f'HDF5:"{os.path.abspath(H5_PATH)}"://SDS/LST'
    print("Trying SDS:", sds_path)
    sds = gdal.Open(sds_path, gdal.GA_ReadOnly)
    print("Open SDS success?", sds is not None)
except RuntimeError as e:
    print("Open SDS error:", e)


Resolved path: C:\Users\ROG\Documents\Termatics\ecostress\ecostress_downloads\ECOv002_L2G_LSTE_39606_011_20250701T045755_0713_01.h5
Exists?      : True
Size (MB)    : 275.032
HDF5 driver present? False
OpenEx error: `ecostress_downloads\ECOv002_L2G_LSTE_39606_011_20250701T045755_0713_01.h5' not recognized as being in a supported file format. It could have been recognized by driver HDF5, but plugin gdal_HDF5.dll is not available in your installation. You may install it with 'conda install -c conda-forge libgdal-hdf5'
Trying SDS: HDF5:"C:\Users\ROG\Documents\Termatics\ecostress\ecostress_downloads\ECOv002_L2G_LSTE_39606_011_20250701T045755_0713_01.h5"://SDS/LST
Open SDS error: `HDF5:"C:\Users\ROG\Documents\Termatics\ecostress\ecostress_downloads\ECOv002_L2G_LSTE_39606_011_20250701T045755_0713_01.h5"://SDS/LST' does not exist in the file system, and is not recognized as a supported dataset name.
